In [4]:
%wandb login 5a76195fbae5605fe5784ebabce86e168ccdfb12

In [3]:
import wandb

In [1]:
import os 
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

# from tensorflow.keras.applications import Densnet
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD, Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import *
from sklearn.preprocessing import LabelEncoder

/home/ubuntu/anaconda3/envs/smartpot/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
BATCH_SIZE = 20
IMAGE_SIZE = 320

In [2]:
root_dir = '/home/lab38/images'
sample_csv = '딸기_정상_수정본.csv'
df = pd.read_csv(os.path.join(root_dir, sample_csv))
df.head()

,image,date,height,width,type,disease,crop,area,grow,risk,points
0,V006_77_0_00_04_01_13_0_b02_20201203_0000_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1167, 'xbr': 2255, 'ybr'..."
1,V006_77_0_00_04_01_13_0_b02_20201203_0001_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 600, 'ytl': 1087, 'xbr': 1628, 'ybr':..."
2,V006_77_0_00_04_01_13_0_b02_20201203_0002_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1217, 'xbr': 2503, 'ybr'..."
3,V006_77_0_00_04_01_13_0_b02_20201203_0003_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 662, 'ytl': 1252, 'xbr': 2129, 'ybr':..."
4,V006_77_0_00_04_01_13_0_b02_20201203_0004_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 370, 'ytl': 884, 'xbr': 1994, 'ybr': ..."


12,13... 으로 되어있는 레이블을 0,1로 바꿔줌  
scikit learn : LabelEncoder

In [3]:
a_en = LabelEncoder()
g_en = LabelEncoder()
d_en = LabelEncoder()

a_en.fit(df['area'])
g_en.fit(df['grow'])
d_en.fit(df['disease'])

df['area'] = a_en.transform(df['area'])
df['grow'] = g_en.transform(df['grow'])
df['disease'] = d_en.transform(df['disease'])
df.head()

,image,date,height,width,type,disease,crop,area,grow,risk,points
0,V006_77_0_00_04_01_13_0_b02_20201203_0000_S01_...,2020-12-03,4032,3024,0,0,4,0,1,0,"[{'xtl': 1003, 'ytl': 1167, 'xbr': 2255, 'ybr'..."
1,V006_77_0_00_04_01_13_0_b02_20201203_0001_S01_...,2020-12-03,4032,3024,0,0,4,0,1,0,"[{'xtl': 600, 'ytl': 1087, 'xbr': 1628, 'ybr':..."
2,V006_77_0_00_04_01_13_0_b02_20201203_0002_S01_...,2020-12-03,4032,3024,0,0,4,0,1,0,"[{'xtl': 1003, 'ytl': 1217, 'xbr': 2503, 'ybr'..."
3,V006_77_0_00_04_01_13_0_b02_20201203_0003_S01_...,2020-12-03,4032,3024,0,0,4,0,1,0,"[{'xtl': 662, 'ytl': 1252, 'xbr': 2129, 'ybr':..."
4,V006_77_0_00_04_01_13_0_b02_20201203_0004_S01_...,2020-12-03,4032,3024,0,0,4,0,1,0,"[{'xtl': 370, 'ytl': 884, 'xbr': 1994, 'ybr': ..."


In [17]:
# df 순서 섞음
shuffled_df = df.sample(frac=1)
# Train : Validation = 0.8 : 0.2 Split
train_len = int(len(df)*0.8)

train_df = shuffled_df[:100]
valid_df = shuffled_df[-100:]

In [19]:
train_gen = ImageDataGenerator(rescale=1./255,
                            rotation_range=20, # 최대 20도까지 회전
                            width_shift_range=0.1, # 최대 x 범위안에서 좌우/상하 이동
                            height_shift_range=0.1,
                            zoom_range=0.2, # 확대 축소 비율,
                            horizontal_flip=True, # 좌우반전
                            vertical_flip=True, # 상하반전
                            fill_mode='nearest')
valid_gen = ImageDataGenerator(rescale= 1. /255.)

In [20]:
img_path = os.path.join(root_dir, '딸기_정상')
train_generator = train_gen.flow_from_dataframe(train_df, 
                                                directory=img_path,
                                               x_col='image',
                                               y_col='area',
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='raw',
                                               batch_size=BATCH_SIZE)
valid_generator = valid_gen.flow_from_dataframe(valid_df,
                                                directory=img_path,
                                               x_col='image',
                                               y_col='area',
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='raw',
                                               batch_size=BATCH_SIZE)

Found 100 validated image filenames.
Found 100 validated image filenames.


sample test model  
- multi classification
- sigle output model

In [23]:
inputs = keras.Input(shape=(320, 320, 3), name="img")
x = layers.Conv2D(16, 3, activation="relu")(inputs)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.3)(x)
d_out = layers.Dense(df['area'].nunique(), name='d', activation='softmax')(x)
model = keras.Model(inputs, d_out, name="test")
model.summary()

Model: "test"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 320, 320, 3)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 318, 318, 16)      448       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 316, 316, 32)      4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 105, 105, 32)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 352800)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 352800)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1128963

In [26]:
model.compile(loss='sparse_categorical_crossentropy', # one hot encoding 안함
             optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [27]:
model.fit(train_generator,
         validation_data=valid_generator,
         epochs=1,
         verbose=1,
         steps_per_epoch = 5) 

5/5 [==============================] - 38s 8s/step - loss: 1.6478 - accuracy: 0.3178 - val_loss: 1.4016 - val_accuracy: 0.2000
